In [ ]:
import pandas as pd
import numpy as np
!pip install -U gensim
import gensim
from gensim.utils import simple_preprocess
from gensim import corpora
from gensim.models import Phrases
from gensim.models import CoherenceModel
from sklearn.manifold import TSNE
import nltk
from nltk.corpus import PlaintextCorpusReader
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import spacy
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

import matplotlib.pyplot as plt

     |████████████████████████████████| 24.1 MB 1.2 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
!pip install pyLDAvis==2.1.2

     |████████████████████████████████| 1.6 MB 10.4 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97738 sha256=c8d318766d2f18099c9709da7a40de7a3711f41f299b6b1013dbe052ad556ca7
  Stored in directory: /root/.cache/pip/wheels/3b/fb/41/e32e5312da9f440d34c4eff0d2207b46dc9332a7b931ef1e89
Successfully built pyLDAvis


In [ ]:
import pyLDAvis
import pyLDAvis.gensim

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [ ]:
#load folders of documents from google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#read in copus
corpus_root = '/content/drive/MyDrive/10-k'
corpus = PlaintextCorpusReader(corpus_root, '.*')

In [ ]:
#number of documents
len(corpus.fileids())

690

In [ ]:
#turn the corpus to dataframe
rows = []
for filename in corpus.fileids():
  rows.append((filename, corpus.raw(filename)))

df = pd.DataFrame(rows, columns=['name', 'text'])
df.head()

,name,text
0,1.txt,We are a leading provider of technology-enable...
1,10.txt,"Golden Nugget Online Gaming, Inc. (formerly kn..."
2,100.txt,This annual report on Form 10-K contains forwa...
3,1001.txt,We are a global designer and manufacturer of h...
4,1002.txt,This Management’s Discussion and Analysis of F...


# noise

In [ ]:
communication_service_noise = 'subscriber million fiscal tv customer medium tds segment percent adjust wireless dish cable dealer spectrum april station merger goodwill license ebitda senior mobile national program local cinema unit satellite vehicle outdoor charter cellular ccoh guarantee indenture ccwh subordinate telecom america chapter legacy sg wireline ihm cco subsidiary intercompany metropcs commercial centric handset ghz ccibv ebita bankruptcy action salary international auction display priority brand foreign series billion broadband network client holding fcc arpu february emmis take data fluent consideration lma digonex percentage contingent topco focus tech nextradio dice gain plus wbls wlib vest quantitative reimbursement recruitment qualify matter except special energy borrower tagstation revolve mostly york amendment pension fm radio facility borrowing gaap lien subscription television consumer production meredith prefer sport stg newsstand bank redeemable dsg raycom living intermediary repack swap rsns publication president standard search multi noncontrolling allrecipes inc capitalize vice announce accrual woman june magazine political broadcast digital affiliation advertiser profit pandemic covid cablevision newsday comcast sky csc aocf real speed estate nbcuniversal lightpath licensing bresnan release pyeongchang clearview theme forma pro exhibition collateralize australian occupancy admission theater absence theatrical phone amc zealand elimination concession australia film video olympics technical currency restrict park voice franchise restructuring residential simplex maintenance duplex message spok way pc investor thermo promise decline device transmitter temporary accretion partially hughes repurchase ftes anticipate variable ntelos pop payroll clinical allocate igo downside minute rgus effort derivative software fiber sprint tower asc go spot page promissory solution research convertible dollar prepaid sling final mhz build wikisoft tmus ntm snr northstar deployment block doj db deadline echostar toll lineup receiver univision stipulation avm removal chat shareholder micro reverse firstmet corp aws activation de user churn lead introduction truecar adjustment oem guaranteed land xm buying taiwan count car parcel forecast auto volume dealerscience subsidy executory daily airline transmission divide promotional affinity glossary engineering automaker certify royalty consult v launch billing incentive click lease traffic impairment charge learn education approximately platform index eps lesser favorable academic wiley university ipo student society characteristic blackwell collect responsible fraud rca bonus annualized definite enrollment cloud round kai constant house publishing supplier ad journal detail contribution unfavorable system engagement implement parscale october initial china adversely engage chinese pronouncement assume crown cloudcommerce pledge consistent difficult name miscellaneous travel filing insurance initiative inception patent venture qualitative creative retrospective clwd instrument raise version concern measurement receivable asu partner secure input office contract consultant award adoption principle xa stage absolute face graph exists guidance adaptive carryforward closing reorganization depend recoverable unrecognized profitability spread indicator schedule cmg division pas thus challenge satisfaction consummate recognizes fasb registrant good topic mainly'

#communication_service_noise = 'subscriber million fiscal tv customer medium tds segment percent adjust wireless dish cable dealer spectrum april station goodwill license senior mobile national program local cinema unit satellite vehicle outdoor charter cellular ccoh guarantee indenture ccwh subordinate telecom america chapter legacy sg wireline ihm cco subsidiary intercompany metropcs commercial centric handset ghz ccibv ebita bankruptcy action salary international auction display priority brand foreign series billion broadband network client holding fcc arpu february emmis take data fluent consideration lma digonex percentage contingent topco focus tech nextradio dice gain plus wbls wlib vest quantitative reimbursement qualify matter except special energy borrower tagstation revolve mostly york amendment pension fm radio facility borrowing gaap lien subscription television consumer production meredith prefer sport stg newsstand bank redeemable dsg raycom living intermediary repack swap rsns publication president standard search multi noncontrolling allrecipes inc capitalize vice announce accrual woman june magazine political broadcast digital affiliation advertiser cablevision newsday comcast sky csc aocf real speed estate nbcuniversal lightpath licensing bresnan release pyeongchang clearview theme forma pro exhibition collateralize australian occupancy admission theater absence theatrical phone amc zealand elimination concession australia film video olympics technical currency restrict park voice franchise residential simplex maintenance duplex message spok way pc investor thermo promise decline device transmitter temporary accretion partially hughes repurchase ftes anticipate variable ntelos pop clinical allocate igo downside minute rgus effort derivative software fiber sprint tower asc go spot page promissory solution research convertible dollar prepaid sling final mhz build wikisoft tmus ntm snr northstar deployment block doj db deadline echostar toll lineup receiver univision stipulation avm removal chat shareholder micro reverse firstmet corp aws activation de user churn lead introduction truecar adjustment oem guaranteed land xm buying taiwan count car parcel forecast auto volume dealerscience subsidy executory daily airline transmission divide promotional affinity glossary engineering automaker certify royalty consult v launch billing incentive click lease traffic impairment charge learn education approximately platform index eps lesser favorable academic wiley university ipo student society characteristic blackwell collect responsible fraud rca bonus annualized definite enrollment cloud round kai constant house publishing supplier ad journal detail contribution unfavorable system engagement implement parscale october initial china adversely engage chinese pronouncement assume crown cloudcommerce pledge consistent difficult name miscellaneous travel filing insurance initiative inception patent venture qualitative creative retrospective clwd instrument raise version concern measurement receivable asu partner secure input office contract consultant award adoption principle xa stage absolute face graph exists guidance adaptive carryforward closing reorganization depend recoverable unrecognized spread indicator schedule cmg division pas thus challenge satisfaction consummate recognizes fasb registrant good topic mainly'


In [ ]:
consumer_discretionary_noise = 'fiscal restaurant store million december game u course park exhibition mattel casino fruit hotel percent franchise student cruise sport north unit location juice comparable grill merchandise franchisees online training billion corporate henglong foreign basin loan target abl specialty volume permian asc stoneridge community veoneer ashford brand hsbc venture hubei member divestiture filament global xcel gaap hedge warehouse asg organic bakken depot automotive device segment restructuring ebitda vehicle rental brazil asu currency bank pension hospitality senior franchisee international merger class royalty production vida pura area ratio vb principally herein program vera bradley leverage territory indirect cpg outlet door systemwide refranchising krispy kreme kk restatement appear filing electronic vision developer doughnut winter ad factory ecommerce inventory shop domestic office opening repurchase sg gross april revolve season distribution motorize titanic towables silversea body subscriber recreational fuel ehg buy decline best artifact passenger onboard recruitment yield drydock six incoming aeg ontents driven phone towable motor caribbean workshop versus digital ship mobile american format european dealer europe september week common january offering compucom maternity xpresspa cvc calm jennifer comp product add employment claim cloud refer vroman twelve edison furniture nation organizational convert purchaser approval fli anderson discuss voting salary profit charge division warrant labor department series prefer spa core retail amendment convertible goodwill category relocation acceptance home dilute aspen concentrate modify kiwifruit xinda usu skypeople june fresh university sichuan waitr prc plastic ton bronco billy pear province basket dubai campus squad lien bite vegetable gourmet apple rmb china enrollment receivable beverage timeshare learn title tree cemetery revpar trust hilton wpt instructor esports funeral adr attendee attend allied space library worldwide interval voucher poker aro aec pt family leasehold classroom participant passport advisory education occupancy room pre v lottery sciplay nascar instant land machine jackpot gnog rsilp access close win social earn nmr borgata concession belterra race aebitda sanction mortar brick participation carrier tra classical tabl label rsi bet postage player landlord shipping october vega la system earnout mail slot customize covid pandemic money food llc girl march advertising fisher rmig igaming beauty point software attribute sim user coolbet internet pet relocate gan remodel supply give barbie fleet petsense salon competition stimulate fifty award optimization calculation operator din toy redemption february platform card capacity gift stk kona industry alexander stoney river goldman sachs black wide knight menu beef ss motorhomes rooftop tennessee accord manufacture bargain investee aluminum angel bar bus steel motorhome awning weekly license adjacent guest check rv oem aftermarket oems travel trailer wholesale holding polaris motorcycle orv snowmobile digit pg schlitterbahn repossess bps cog whole orvs side winterfest wfcdf visit amusement pas indian tcf cedar ptp concessionaire king sheffield eppl pontoon synchrony inaugural immersive boat caput water spending gate attendance retirement swap nm warranty attraction redeem admission shipment'

#consumer_discretionary_noise = 'fiscal restaurant store million december game u course park exhibition mattel casino fruit hotel percent franchise student cruise sport north unit location juice comparable grill merchandise franchisees online training billion corporate henglong foreign basin loan target abl specialty volume permian asc stoneridge community veoneer ashford brand hsbc venture hubei member divestiture filament global xcel gaap hedge warehouse asg organic bakken depot automotive device segment vehicle rental brazil asu currency bank pension hospitality senior franchisee international class royalty production vida pura area ratio vb principally herein program vera bradley leverage territory indirect cpg outlet door systemwide refranchising krispy kreme kk restatement appear filing electronic vision developer doughnut winter ad factory ecommerce inventory shop domestic office opening repurchase sg april revolve season distribution motorize titanic towables silversea body subscriber recreational fuel ehg buy decline best artifact passenger onboard recruitment yield drydock six incoming aeg ontents driven phone towable motor caribbean workshop versus digital ship mobile american format european dealer europe september week common january offering compucom maternity xpresspa cvc calm jennifer comp product add employment claim cloud refer vroman twelve edison furniture nation organizational convert purchaser approval fli anderson discuss voting charge division warrant labor department series prefer spa core retail amendment convertible goodwill category relocation acceptance home dilute aspen concentrate modify kiwifruit xinda usu skypeople june fresh university sichuan waitr prc plastic ton bronco billy pear province basket dubai campus squad lien bite vegetable gourmet apple rmb china enrollment receivable beverage timeshare learn title tree cemetery revpar trust hilton wpt instructor esports funeral adr attendee attend allied space library worldwide interval voucher poker aro aec pt family leasehold classroom participant passport advisory education occupancy room pre v lottery sciplay nascar instant land machine jackpot gnog rsilp access close win social earn nmr borgata concession belterra race aebitda sanction mortar brick participation carrier tra classical tabl label rsi bet postage player landlord shipping october vega la system earnout mail slot customize covid pandemic money food llc girl march advertising fisher rmig igaming beauty point software attribute sim user coolbet internet pet relocate gan remodel supply give barbie fleet petsense salon competition stimulate fifty award optimization calculation operator din toy redemption february platform card capacity gift stk kona industry alexander stoney river goldman sachs black wide knight menu beef ss motorhomes rooftop tennessee accord manufacture bargain investee aluminum angel bar bus steel motorhome awning weekly license adjacent guest check rv oem aftermarket oems travel trailer wholesale holding polaris motorcycle orv snowmobile digit pg schlitterbahn repossess bps cog whole orvs side winterfest wfcdf visit amusement pas indian tcf cedar ptp concessionaire king sheffield eppl pontoon synchrony inaugural immersive boat caput water spending gate attendance retirement swap nm warranty attraction redeem admission shipment'


In [ ]:
consumer_staples_noise = 'fiscal million tax store september charge acquisition pharmacy adjust week earnings credit billion august currency impairment customer benefit facility beverage income beauty rate fifty fy foreign segment retail senior keurig lemon real seaboard sugar agribusiness company brazil energy drink pack april group monster brazilian affiliate ebit fertilizer merchandising carton avocado pork farmer bottler case orange pet milling packinghouse promotional grower bunge distributor estate translation volume ethanol oil grain commodity fresh content coty quarter fragrance live travel fourth luxury cigarette tobacco makeup hair region jennie skin younique ghd nicotine east galleria middle aurora africa europe lauder estée redeemable exercise social burberry investment fda euro october gaap measure brand covid june divestiture goodwill reconciliation pandemic warrant consumer auction battery gankit user value energizer auto bid website rule arnold twelve fair november job commerce penny versus internet rb action leverage mix special ratio phase rise add com accompany act small security constant integration flavor exchange care dollar pension non saving program snyder lance defer amend campbell sgm prefer close decline pre postretirement hershey bolthouse chip outlook mark series reimbursement european priority unsecured carrot wellness loa class mauna kelsen soup unit snack initiative realignment february associate trademark loan secure franchise three quest envisionrx dilute tra wba mining opening yogurt leaf aspen project atkins ftc medicare drug corizona rocky mountain aid franchisees fred cm prescription revolver rite factory location merger client restate front ebitda lifo tranche item nutrition ag supplier entity chestnut settlement carbohydrate specify raw comprehensive lorain military martin ingredient comparable team short kitchen independent insignificant individually adm sweetener domestic shandong planning refine organizational deca margin bank restructuring oilseed global food eps solution revolve fulfillment amortization december depreciation cooperative center free channel technology covenant nut wine mortgage progold apron limitation capitalize recipe software refund arlington engagement ipo refinance member linden card blue thousand convertible meal driven exclude corn ifresh march employee chicken offering wholesale trait broiler nym feature tyler poultry home goodwheat feed stock restaurant field marketable occupancy milestone hourly hb trial mask chick head arcadia legacy breeder pound bird grant research hemp asu license texas crop wheat agriculture land box processing cattle alico county grove magnolia rloc acre dispensary inc contract strengthen dakota borrow star angel life golden ridge scale v irs modern maintenance native advantage polk soybean florida day solid sugarcane citrus agricultural fruit china south harvest farm government opportunity system pod coffee brewer acer truncatum canada hot accessory kold warranty agreement cup registration investor bevyz brewing lease investigation iced rou expedient practical hlch makingorg inception nervonic hkfw look forward officer'

#consumer_staples_noise = 'fiscal million tax store september charge pharmacy adjust week earnings credit billion august currency impairment customer benefit facility beverage beauty rate fifty fy foreign segment retail senior keurig lemon real seaboard sugar agribusiness company brazil energy drink pack april group monster brazilian affiliate fertilizer merchandising carton avocado pork farmer bottler case orange pet milling packinghouse promotional grower bunge distributor estate translation volume ethanol oil grain commodity fresh content coty quarter fragrance live travel fourth luxury cigarette tobacco makeup hair region jennie skin younique ghd nicotine east galleria middle aurora africa europe lauder estée redeemable exercise social burberry investment fda euro october gaap measure brand june divestiture goodwill reconciliation pandemic warrant consumer auction battery gankit user value energizer auto bid website rule arnold twelve fair november job commerce penny versus internet rb action leverage mix special ratio phase rise add com accompany act small security constant integration flavor exchange care dollar pension non saving program snyder lance defer amend campbell sgm prefer close decline pre postretirement hershey bolthouse chip outlook mark series reimbursement european priority unsecured carrot wellness loa class mauna kelsen soup unit snack initiative realignment february associate trademark loan secure franchise three quest envisionrx dilute tra wba mining opening yogurt leaf aspen project atkins ftc medicare drug corizona rocky mountain aid franchisees fred cm prescription revolver rite factory location merger client restate front lifo tranche item nutrition ag supplier entity chestnut settlement carbohydrate specify raw comprehensive lorain military martin ingredient comparable team short kitchen independent insignificant individually adm sweetener domestic shandong planning refine organizational deca margin bank oilseed global food eps solution revolve fulfillment amortization december depreciation cooperative center free channel technology covenant nut wine mortgage progold apron limitation capitalize recipe software refund arlington engagement ipo refinance member linden card blue thousand convertible meal driven exclude corn ifresh march employee chicken offering wholesale trait broiler nym feature tyler poultry home goodwheat feed stock restaurant field marketable occupancy milestone hourly hb trial mask chick head arcadia legacy breeder pound bird grant research hemp asu license texas crop wheat agriculture land box processing cattle alico county grove magnolia rloc acre dispensary inc contract strengthen dakota borrow star angel life golden ridge scale v irs modern maintenance native advantage polk soybean florida day solid sugarcane citrus agricultural fruit china south harvest farm government opportunity system pod coffee brewer acer truncatum canada hot accessory kold warranty agreement cup registration investor bevyz brewing lease investigation iced rou expedient practical hlch makingorg inception nervonic hkfw look forward officer'


In [ ]:
energy_noise = 'million gas vessel natural company coal march december ton trust charter successor royalty warrant predecessor product segment derivative stock adjust percent pool apache site consol altus reserve mine billion fuel unit acquisition approximately brigham settlement oasis omp credit murphy connection october mp international haymaker bonus owl ebitdax contribute affiliate discretionary couple elevation free principally cement lynden producer llog afe gather mineral commodity distribution nymex crude prove hedge ipo boe ngl basin volume revolve content liquid ngls partnership transportation midstream barrel goodwill ebitda depletion asphalt ergon pbf chemical east terminalling rpc rockwater cactus con internal pbfx consolidated express domestic cimarron avalon see star trucking ppp specification leverage disposition second vitol adler completion restate affirm water oilfield rig solution storage profit west tra pipeline hour customer terminal gross oklahoma asu infrastructure throughput loan drilling voyage spl vlgcs drydock rhino helios appalachia cqp spot dynaenergetics met train liquefaction tce nobelclad oaktown illinois eastern osg alaskan flag overseas tanker spa sabine pas capex survey bp lpg northern mining central lng venture fleet senior kml individually cbm none ebda pa va division live reportable idle double mvc barnett summit ohio tioga colombia bailey power cnx smp nar tmpl bison cochin netback acordionero comparison mcfe marcellus earnings bcfe mcf utica gathering system variance shale freight eqm circle motor retail fiscal wpz eqt acmp mvp olefin geismar condor miej recovery williams investor midwest upper cst hawk dm absence omnibus sixth asia fcx transco mcmoran unfavorable wholesale ethane trustee merger pre rating partner bridge expansion compressor compression allowance secure san mateo package third redemption valuation indenture construction treat salt waha flare late electrical costless century purchaser drop lift delineation horsepower widen extinguishment money fabricate csi rental black fabrication contract delaware plant exploration permian continent processing bbl september lien underlie xto petrogress ii devon alpine npi sovereign borrowing gain program ca spending hunt liquidation overhead traios good absolute standard kansa page option tmrc epic coma acreage fresh start emergence twelve combine camber conveyance v mboe eagle ford prefer dd april table degree cretic input capitalize bearing sprague residual kildair trade eurocurrency measurement amortize exposure temperature idr resolute york prepaid excise winter observable unamortized destruction multicurrency float reset distillate tube coil inventory heating refine margin pik swap handle unrealized instrument update project growth capacity equivalent effective july mountain section nil grant forward mont belvieu commercial retrospective unused stream separate code physical oneok debtor website www principle accepted conform prospective partial rocky guidance fasb severance bankruptcy look presentation mbbl please dividend region unproved expiration egypt extend sea zion australia exercise america february whitehead mj exercisable argentina prospectus worldwide registration duration regulation kitimat assurance znwah znwaj znwai znwad supplement dspp israel participant bond recompletion mb north share split agent termination canada divestiture license gulf'

#energy_noise = 'million gas vessel natural company coal march december ton trust charter successor royalty warrant predecessor product segment derivative stock adjust percent pool apache site consol altus reserve mine billion fuel unit approximately brigham settlement oasis omp credit murphy connection october mp international haymaker bonus owl ebitdax contribute affiliate discretionary couple elevation free principally cement lynden producer llog afe gather mineral commodity distribution nymex crude prove hedge ipo boe ngl basin volume revolve content liquid ngls partnership transportation midstream barrel goodwill depletion asphalt ergon pbf chemical east terminalling rpc rockwater cactus con internal pbfx consolidated express domestic cimarron avalon see star trucking ppp specification leverage disposition second vitol adler completion restate affirm water oilfield rig solution storage west tra pipeline hour customer terminal gross oklahoma asu infrastructure throughput loan drilling voyage spl vlgcs drydock rhino helios appalachia cqp spot dynaenergetics met train liquefaction tce nobelclad oaktown illinois eastern osg alaskan flag overseas tanker spa sabine pas capex survey bp lpg northern mining central lng venture fleet senior kml individually cbm none ebda pa va division live reportable idle double mvc barnett summit ohio tioga colombia bailey power cnx smp nar tmpl bison cochin netback acordionero comparison mcfe marcellus bcfe mcf utica gathering system variance shale freight eqm circle motor retail fiscal wpz eqt acmp mvp olefin geismar condor miej recovery williams investor midwest upper cst hawk dm absence omnibus sixth asia fcx transco mcmoran unfavorable wholesale ethane trustee merger pre rating partner bridge expansion compressor compression allowance secure san mateo package third redemption valuation indenture construction treat salt waha flare late electrical costless century purchaser drop lift delineation horsepower widen extinguishment money fabricate csi rental black fabrication contract delaware plant exploration permian continent processing bbl september lien underlie xto petrogress ii devon alpine npi sovereign borrowing gain program ca spending hunt liquidation overhead traios good absolute standard kansa page option tmrc epic coma acreage fresh start emergence twelve combine camber conveyance v mboe eagle ford prefer dd april table degree cretic input capitalize bearing sprague residual kildair trade eurocurrency measurement amortize exposure temperature idr resolute york prepaid excise winter observable unamortized destruction multicurrency float reset distillate tube coil inventory heating refine margin pik swap handle unrealized instrument update project capacity equivalent effective july mountain section nil grant forward mont belvieu commercial retrospective unused stream separate code physical oneok debtor website www principle accepted conform prospective partial rocky guidance fasb severance look presentation mbbl please region unproved expiration egypt extend sea zion australia exercise america february whitehead mj exercisable argentina prospectus worldwide registration duration regulation kitimat assurance znwah znwaj znwai znwad supplement dspp israel participant bond recompletion mb north share split agent termination canada divestiture license gulf'


In [ ]:
healthcare_noise = 'claim fiscal million december march covid care warrant scilex research clinical premium backlog pandemic investment segment medical royalty candidate product tax credit trial growth direct june mostly ii september sofusa insurance reserve membership manage earn point actuarial casualty mlr favorable utilization puerto benefit rico unfavorable insured behavioral hip percent pbm magellan reinsurance hif eap variance ibnr statute intersegment mainly pharmacy member repurchase ratio specialty content health healthcare medicaid medicare living community receipt snh cbd cold combination psa return rpi partnership transitional nerve ipi axogen biohaven cystic fibrosis pennant old tysabri analyst cardinal affiliate tecfidera side mature imbruvica brand pharmaceutical skilled stage senior legacy decline facility recognizes occupancy gaap billion nursing lease acquisition day gross inventory milestone satisfied licensee prepayment pump reimbursement infusion pdl july oncology precision glumetza indemnification amendment chase janssen harm cambium landlord partially zipsor equipment consumables nucynta lazanda sequence implementation ambulatory disposable simponi median device payer section test jurisdiction payor loan patient cantor clinic billing exercise instrument approximately payors patent revolve ztlido lidocaine topical kimberly clark performance conditional antibody virttu purchaser rtx belumosudil tnk car adxs celularity whether derivative oaktree nih rare adc bdl therapy bankruptcy lenzilumab tumor ifabotuzumab october license study indenture asc center home hospice census lumateperone aptevo metric rapivab galidesivir billable iti relief hour ixinity bcx placebo hhs attack agency hedge episode admission analytical extinguishment peramivir medexus stimulus hae kentucky indicator act platform wage pas illinois client chairman president provision consists practice amih executive profitability emerge lien impute inflation comply taxed mevion gamma developer knife system fy regional pbrt dermatology optical reshape symbion derive geographic avail silicon valley fluctuate distributor pressure surgical contract job physician impairment profit stand alone surgery asu franchise ghs animal human espp liver receivership cell vivo thousand bioprinter vitro novogen orphan charity livestock university bridge development bioprinted bioprinting designation academic observe affiliation organovo functional samsara biology advisory promissory customer topic venture grant tissue joint collaborative shelf inc convertible disease ebitda collaboration hsa wageworks oil disruption custodial hsas cdd lcd sleep opening solsys failure cdbs translation amca bruker resmed saas interchange gas bsi depository ato mask healthequity acceptance swk organic card partner currency unit foreign margin pension laboratory network goodwill reimburse iii authorization redemption holdco elsewhere across opco headcount lack pocket redeem recruitment atrm kb definite marketability argus image digirad orion dca offering gerber diagnostic promotional sg detail phase v cpm abaloparatide brook case mark calendar observable chapter approximate rloc orthopedic maxim biologics elacestrant propel medusa overlord fuse ambio patch sc tymlos sinuva successor sintu wellcare il sinus reeg nc amniotic christopher nbmj implant drug'

#healthcare_noise = 'claim fiscal million december march care warrant scilex research clinical premium backlog pandemic segment medical royalty candidate product tax credit trial direct june mostly ii september sofusa insurance reserve membership manage earn point actuarial casualty mlr favorable utilization puerto benefit rico unfavorable insured behavioral hip percent pbm magellan reinsurance hif eap variance ibnr statute intersegment mainly pharmacy member repurchase ratio specialty content health healthcare medicaid medicare living community receipt snh cbd cold combination psa return rpi partnership transitional nerve ipi axogen biohaven cystic fibrosis pennant old tysabri analyst cardinal affiliate tecfidera side mature imbruvica brand pharmaceutical skilled stage senior legacy decline facility recognizes occupancy gaap billion nursing lease day gross inventory milestone satisfied licensee prepayment pump reimbursement infusion pdl july oncology precision glumetza indemnification amendment chase janssen harm cambium landlord partially zipsor equipment consumables nucynta lazanda sequence implementation ambulatory disposable simponi median device payer section test jurisdiction payor loan patient cantor clinic billing exercise instrument approximately payors patent revolve ztlido lidocaine topical kimberly clark performance conditional antibody virttu purchaser rtx belumosudil tnk car adxs celularity whether derivative oaktree nih rare adc bdl therapy bankruptcy lenzilumab tumor ifabotuzumab october license study indenture asc center home hospice census lumateperone aptevo metric rapivab galidesivir billable iti relief hour ixinity bcx placebo hhs attack agency hedge episode admission analytical extinguishment peramivir medexus stimulus hae kentucky indicator act platform wage pas illinois client chairman president provision consists practice amih executive emerge lien impute inflation comply taxed mevion gamma developer knife system fy regional pbrt dermatology optical reshape symbion derive geographic avail silicon valley fluctuate distributor pressure surgical contract job physician impairment profit stand alone surgery asu franchise ghs animal human espp liver receivership cell vivo thousand bioprinter vitro novogen orphan charity livestock university bridge development bioprinted bioprinting designation academic observe affiliation organovo functional samsara biology advisory promissory customer topic venture grant tissue joint collaborative shelf inc convertible disease ebitda collaboration hsa wageworks oil disruption custodial hsas cdd lcd sleep opening solsys failure cdbs translation amca bruker resmed saas interchange gas bsi depository ato mask healthequity acceptance swk organic card partner currency unit foreign margin pension laboratory network goodwill reimburse iii authorization redemption holdco elsewhere across opco headcount lack pocket redeem recruitment atrm kb definite marketability argus image digirad orion dca offering gerber diagnostic promotional sg detail phase v cpm abaloparatide brook case mark calendar observable chapter approximate rloc orthopedic maxim biologics elacestrant propel medusa overlord fuse ambio patch sc tymlos sinuva successor sintu wellcare il sinus reeg nc amniotic christopher nbmj implant drug'


In [ ]:
industrials_noise = 'million qtr fiscal segment st december refinery restructuring product foreign industrial contract defense plan loan fh fuel energy aerospace september special patent thousand commercial march solution power equipment warrant lease wind blade july data diversified w dealer ev distributor cmw webbank transformation wse mold correspond raw op hr hardware premium mhps ratio arc accessory table rpo consideration care contingent developer professional startup commodity intangible residential gaap emea project manufacturing goodwill sg profit earnings ebitda construction currency client insurance north percent guarantee inventory covid adjust acquisition october content pension growth mexican reserve office llc recur railway thermal advanced margin acoustical november carload rationalization nonwovens ecr spig flex peso wheel franchisees kcsm mtm kforce august graphite electrode gain maslow initiative merger technical environmental coal filtration joint volume pandemic kc revolve organic decline discontinue senior second billion staff claim utc pratt whitney max structure kopple pb climate control refer quantity airplane cumulative recreation appear core sikorsky bd aircraft delivery wave catch ambulance jet otis subsea sense hybrid bgs april boeing engine program space military production divestiture system bus backlog aftermarket trailer tractor expedite pool container driven principally credit truckload jbt logistics partly operator owner salvage test highway robuschi trucking relates transportation incentive right brokerage aerotech manage foodtech transactional subsidiary freight intermodal driver mile railcar wage surcharge fleet group rental shipment salary carrier technology portfolio passur licensing enforcement access litigation franchise atv trial fmtv floor bird blue licensee inventor airport performance lime name completion dod gilbert onsite weidt bridge smlat dealership royalty government license inc duty june vehicle engineering software quarter warranty direct exchange orbital promissory principal cui benelux investor common pvbj avention dbcc hydrogen pursuant dp dun bradstreet firstfire divest enertec gaspt institutional wdco pride jm bond satisfaction microphase lighthouse grow atm uk conversion convertible holder america qualify trading detail offering january marketing postretirement series th saving rd nd fluid valve unaudited reliability pump colfax okc circor actuation cf beat provisional gilti hq close wide infrastructure finance search tti walmart amazon training raven executive creek ic film severance axon practice cedar ppas gendrive aerostar genkey vest ppa autonomy upfront learn leaseback gensure cell global adoption division topic month option atg cpg lead monthly agent particularly secure intellitronix aci tower integrity aluminum overall bulb bar collateral indebtedness rough terrain thereto exception director manufacture demag platform twelve leave alloy sterling lien aerial corporation pricing light indenture crane abl installation good awp stockholder tfs terex day mp csx cfr odac hill germany libya exone track gmbh fra esa oman velocity cpi qatar gersthofen east arrival iraq lbm headquarters canaccord oppenheimer csxt yard ptc gnc cadogans brazil binder printing machine post middle train kingdom reimbursable print injury retirement railroad rating look car research'

#industrials_noise = 'million qtr fiscal segment st december refinery product foreign industrial contract defense plan loan fh fuel energy aerospace september special patent thousand commercial march solution power equipment warrant lease wind blade july data diversified w dealer ev distributor cmw webbank transformation wse mold correspond raw op hr hardware premium mhps ratio arc accessory table rpo consideration care contingent developer professional startup commodity intangible residential gaap emea project manufacturing goodwill sg construction currency client insurance north percent guarantee inventory djust october content pension growth mexican reserve office llc recur railway thermal advanced margin acoustical november carload rationalization nonwovens ecr spig flex peso wheel franchisees kcsm mtm kforce august graphite electrode gain maslow initiative technical environmental coal filtration joint volume pandemic kc revolve organic decline discontinue senior second billion staff claim utc pratt whitney max structure kopple pb climate control refer quantity airplane cumulative recreation appear core sikorsky bd aircraft delivery wave catch ambulance jet otis subsea sense hybrid bgs april boeing engine program space military production divestiture system bus backlog aftermarket trailer tractor expedite pool container driven principally credit truckload jbt logistics partly operator owner salvage test highway robuschi trucking relates transportation incentive right brokerage aerotech manage foodtech transactional subsidiary freight intermodal driver mile railcar wage surcharge fleet group rental shipment salary carrier technology portfolio passur licensing enforcement access litigation franchise atv trial fmtv floor bird blue licensee inventor airport performance lime name completion dod gilbert onsite weidt bridge smlat dealership royalty government license inc duty june vehicle engineering software quarter warranty direct exchange orbital promissory principal cui benelux investor common pvbj avention dbcc hydrogen pursuant dp dun bradstreet firstfire divest enertec gaspt institutional wdco pride jm bond satisfaction microphase lighthouse grow atm uk conversion convertible holder america qualify trading detail offering january marketing postretirement series th saving rd nd fluid valve unaudited reliability pump colfax okc circor actuation cf beat provisional gilti hq close wide infrastructure finance search tti walmart amazon training raven executive creek ic film severance axon practice cedar ppas gendrive aerostar genkey vest ppa autonomy upfront learn leaseback gensure cell global adoption division topic month option atg cpg lead monthly agent particularly secure intellitronix aci tower integrity aluminum overall bulb bar collateral indebtedness rough terrain thereto exception director manufacture demag platform twelve leave alloy sterling lien aerial corporation pricing light indenture crane abl installation good awp stockholder tfs terex day mp csx cfr odac hill germany libya exone track gmbh fra esa oman velocity cpi qatar gersthofen east arrival iraq lbm headquarters canaccord oppenheimer csxt yard ptc gnc cadogans brazil binder printing machine post middle train kingdom reimbursable print injury retirement railroad rating look car research'


In [ ]:
it_noise = 'fiscal december million loan subscription kodak software september january june license inventory gaap adjust acquisition platform non billion bitcoin currency solution europe maintenance asc cloud asu xerox thousand earnings bank november licensee automotive chain october mgt modem game roccat etailz hp gamers sonic packard hewlett samsung compute rf fye reference console microphone intellectual nominal mc category recorder forecast nxp wireless jv framework supply responsibility litigation ip licensing percentage image visa tape headset v volume minimum camera nm royalty segment repurchase march covid marketable restructuring distributor content share korean fab gain subsidiary novitex common secure prefer intercompany others nasdaq oled restatement offering residual llps itps exchangeable live msa index shift concrete h inch american envestnet termination payonline korea eliminates rmb display merchant extinguishment power ebitda foreign series derivative senior manufacturing processing warrant impairment hedge client ge operational premise salary pro headstrong sow evidence qad inflow dragon briefcam transcription headcount appliance esw forego gpm pre genpact kandy combine computex indian microwave ehr qumu appraisal objective early india system host wage medical approximately division healthcare professional personnel constant sg video travel billing pandemic fy decline point refer outsource fuji contribution seat print productivity film transfer money sag abl color annuity rd entry inkjet saving conduent global document plate signing api message actuarial agent mid pension contract bpo usage center mobile separation retirement consumer midcap smartvoice sphg forma iwco saba planetsoft cordless ebix stockholder smarthome cross soundchip moduslink organization property adam pertain protective amortization flash cerberus parent allocation expansion iia trail depreciation reallocation drive mainly unified july insurance absolute retention recur communication except convertible enterprise guidance research april bitcoins security low adversely quarter transitional predecessor mining il spending successor maturity miner block program consignment blockchain cisco pending ols remote assembly duo allen cypress fluctuate conflict cftc foundry network dividend unit award update shipment product warranty semiconductor production wafer gross ii enertec ya profit activation combination micronet zoom seda mercantile electronics input prc vie techxtend coolisys synchronoss lifeboat competitive motorola razorsight intralinks health quantenna meydan phantom advertiser verizon audit shall adoption agree february library distribution amendment amend registration vendor fasb divestiture lease bill member participation receivables consultant greensky inception attributable average g contingent classification fcr classify libor dsc createapp disclose mark spv enough promotional brewery origination separate brewing hnt evaluates daily deposit partner warehouse funding entity certificate retrospective escrow portfolio guarantee instrument private driven growth measure mutual fortress broadridge medtech consist bridgeline right spangenberg opportunity manager grant orthophoenix outside get corecard add unbound msp sub reconciliation overhead migrate cyberfone probable abandonment patent dilute ibm legacy size proxy consult acquire trade investor marketing goodwill saas intangible digital broadcast acacia advertising medium filter accrue forward mfc medrecycler act rhode ebc aiware implement alldigital ri xumanii solar bdi bayhawk automatically section accelerator broadcasting shelter compose radio jdl beer registrant bridge placement bandwidth raise device island agency register cognitive ipo ai look corp merger inc comparable versus international'

#it_noise = 'fiscal december million loan subscription kodak software september january june license inventory gaap adjust platform non billion bitcoin currency solution europe maintenance asc cloud asu xerox thousand earnings bank november licensee automotive chain october mgt modem game roccat etailz hp gamers sonic packard hewlett samsung compute rf fye reference console microphone intellectual nominal mc category recorder forecast nxp wireless jv framework supply responsibility litigation ip licensing percentage image visa tape headset v volume minimum camera nm royalty segment repurchase march covid marketable distributor content share korean fab gain subsidiary novitex common secure prefer intercompany others nasdaq oled restatement offering residual llps itps exchangeable live msa index shift concrete h inch american envestnet termination payonline korea eliminates rmb display merchant extinguishment power foreign series derivative senior manufacturing processing warrant impairment hedge client ge operational premise salary pro headstrong sow evidence qad inflow dragon briefcam transcription headcount appliance esw forego gpm pre genpact kandy combine computex indian microwave ehr qumu appraisal objective early india system host wage medical approximately division healthcare professional personnel constant sg video travel billing pandemic fy point refer outsource fuji contribution seat print productivity film transfer money sag abl color annuity rd entry inkjet saving conduent global document plate signing api message actuarial agent mid pension contract bpo usage center mobile separation retirement consumer midcap smartvoice sphg forma iwco saba planetsoft cordless ebix stockholder smarthome cross soundchip moduslink organization property adam pertain protective amortization flash cerberus parent allocation expansion iia trail depreciation reallocation drive mainly unified july insurance absolute retention recur communication except convertible enterprise guidance research april bitcoins security low adversely quarter transitional predecessor mining il spending successor maturity miner block program consignment blockchain cisco pending ols remote assembly duo allen cypress fluctuate conflict cftc foundry network dividend unit award update shipment product warranty semiconductor production wafer gross ii enertec ya activation combination micronet zoom seda mercantile electronics input prc vie techxtend coolisys synchronoss lifeboat competitive motorola razorsight intralinks health quantenna meydan phantom advertiser verizon audit shall adoption agree february library distribution amendment amend registration vendor fasb divestiture lease bill member participation receivables consultant greensky inception attributable average g contingent classification fcr classify libor dsc createapp disclose mark spv enough promotional brewery origination separate brewing hnt evaluates daily deposit partner warehouse funding entity certificate retrospective escrow portfolio guarantee instrument private driven growth measure mutual fortress broadridge medtech consist bridgeline right spangenberg opportunity manager grant orthophoenix outside get corecard add unbound msp sub reconciliation overhead migrate cyberfone probable abandonment patent dilute ibm legacy size proxy consult trade investor marketing goodwill saas intangible digital broadcast acacia advertising medium filter accrue forward mfc medrecycler act rhode ebc aiware implement alldigital ri xumanii solar bdi bayhawk automatically section accelerator broadcasting shelter compose radio jdl beer registrant bridge placement bandwidth raise device island agency register cognitive ipo ai look corp inc comparable versus international'


In [ ]:
materials_noise = 'million fiscal mill ounce tax facility percent quarter gold paper product income exploration fourth segment june sustain copper full mineral march ebitda leach pension stream mine reclamation verso december pulp huntsman plant revolve corporation closing asbestos retirement adjust avonmore roll live program forge cement kosmos cast titanium dioxide post twelve gas advanced early separation see wallboard spin asw pori air european restructuring international additive postretirement content oil respectively gross eastman unusual core gasification return eastmanlogoa jpg site decline afp electronic ci europe america solar airlaid eps amend paste timberland solutia petroleum leverage mtm allocation forth specialty action acetate translation precious incident currency gaap region ferro ebit foreign thousand metal profit coal senior lease lumber loan water mpr energy admts realization functional admt drainage strong mitigate solution ppp downtime nbsk emea pressure swap denominate list german receivables produce infrastructure revolver tile ikonics celgar pipe fiber euro canadian attributable coating wood customer ton goodwill growth reportable map oz versus silver operator percentage cpg spg pcg billion margin variance mlbs organic pretax ibt ship milligan mount half viejo marketable cortez pueblo calendar spot unremitted royalty sg consumer unfavorable convertible stage covid ore pad grade stockpile remediation january yanacocha australia newmont merian boddington phoenix cc tanami canyon south carlin ahafo throughput twin provisional proven hijau batu october kalgoorlie akyem pound mining claim performance favorable tissue caprolactone driven pro forma munhall galvanize pine musa packaging pavement ingevity tank oilfield industrial tube retail sb bonus cto category seven storage intellectual stainless chemical technology earn paperboard ii v wisconsin luke rapid november duluth stevens pixelle print proxy graphic audit deficiency comparison independent suffer become recur concern able idle inception auditor assurance androscoggin even explanatory commercially perhaps describe possibly stockholder abl closure machine coat raise successor july predecessor fresh depletion bankruptcy chapter wickliffe newpage petition debtor emergence maine start catalog dip hydroelectric magazine possession lstc bucksport reorganization mountain paramount permit grassy sleeper cpa submit land oregon administration offering blm nevada revise agency feasibility budget notice study poo propose wa ausenco comment dogami completeness county drill malheur register hold stock fee common compensation weight shareholder dilute accepted basic grant anniversary web transfer prepared employment proceed repay adoption filing officer split enable consultant divide execution presently assay lose marcus institute agent forward elsewhere option april consult nil look'

#materials_noise = 'million fiscal mill ounce tax facility percent quarter gold paper product income exploration fourth segment june sustain copper full mineral march leach pension stream mine reclamation verso december pulp huntsman plant revolve corporation closing asbestos retirement adjust avonmore roll live program forge cement kosmos cast titanium dioxide post twelve gas advanced early separation see wallboard spin asw pori air european international additive postretirement content oil respectively gross eastman unusual core gasification return eastmanlogoa jpg site decline afp electronic ci europe america solar airlaid eps amend paste timberland solutia petroleum leverage mtm allocation forth specialty action acetate translation precious incident currency gaap region ferro foreign thousand metal coal senior lease lumber loan water mpr energy admts realization functional admt drainage strong mitigate solution ppp downtime nbsk emea pressure swap denominate list german receivables produce infrastructure revolver tile ikonics celgar pipe fiber euro canadian attributable coating wood customer ton goodwill growth reportable map oz versus silver operator percentage cpg spg pcg billion margin variance mlbs organic pretax ibt ship milligan mount half viejo marketable cortez pueblo calendar spot unremitted royalty sg consumer unfavorable convertible stage covid ore pad grade stockpile remediation january yanacocha australia newmont merian boddington phoenix cc tanami canyon south carlin ahafo throughput twin provisional proven hijau batu october kalgoorlie akyem pound mining claim performance favorable tissue caprolactone driven pro forma munhall galvanize pine musa packaging pavement ingevity tank oilfield industrial tube retail sb bonus cto category seven storage intellectual stainless chemical technology earn paperboard ii v wisconsin luke rapid november duluth stevens pixelle print proxy graphic audit deficiency comparison independent suffer become recur concern able idle inception auditor assurance androscoggin even explanatory commercially perhaps describe possibly stockholder abl closure machine coat raise successor july predecessor fresh depletion bankruptcy chapter wickliffe newpage petition debtor emergence maine start catalog dip hydroelectric magazine possession lstc bucksport reorganization mountain paramount permit grassy sleeper cpa submit land oregon administration offering blm nevada revise agency feasibility budget notice study poo propose wa ausenco comment dogami completeness county drill malheur register hold stock fee common compensation weight shareholder dilute accepted basic grant anniversary web transfer prepared employment proceed repay adoption filing officer split enable consultant divide execution presently assay lose marcus institute agent forward elsewhere option april consult nil look'


In [ ]:
real_estate_noise = 'million lease newmark hotel property partnership development fiscal venture tenant segment bgc september land homesale senior realogy center holding ebitda commercial group jv community sugar ffo rent reit oil joint prefer weight building dividend unit street mortgage renewal locate expand termination construction washington parking outdoor libor credit advertising repurchase held six five york fdx newly unrealized columbia wireless frp lessee mainly series stock security rental square noi foot expansion unconsolidated program industrial content nrt franchisees relocation rfg shopping cartus amendment refinance independent securitization side regional initiative affinity franchisee forma global translation cbre overview telford trg phh crisis remeasurement solution mall volume brokerage franchise agent royalty commission home driven dollar term distribution funding mffo november receivable amarra direct affiliate merger barton internalization heb kingwood drive mcfa resident developed day straight receivables acl pnc lantana santal block reinvestment entertainment member adoption asc austin master project hedge creek lot family trust multifamily phase store covid apartment asu gas water production mineral crop farm principally resort undeveloped ccsid almond pistachio trcc great ranch class produce barrel valencia approval highlight holder cfd grapevine eleven lennar rlc orchard shipyard candlestick acre entitle gateway park francisco san residential percent homesites bbxre renin bbx florida location club altman beach hospitality watersound profit pier hurricane crossing michael watercolor vacation marina origin fl sweet supplier colonial elegance northwest homesite joe southwood bvh bankruptcy pandemic county timber mae comparable fannie department gse originate mac freddie room epus competitive forbearance lodge revpar guest labor penetration adr renovation concern ff payable pursuit live compact gift transient raise disposal nasdaq beverage food cantor origination fpus unsecured caesar vici cplv iii second leverage abr propco sign execute deferral grid gla loudoun carillon extension downtown default progress la cmb quarterfield shoppes harrah east essential scope bill ceoc concession retail spread lien address redevelopment abatement multi golf shareholder exclude tax gross local margin rate exchange transaction lasalle reimburse professional asia hff msr emea banking demand consult america drove notable comparative country post pronouncement size geographically nearly etr singapore growth incremental currency pacific earn client advisory foreign billion reimbursement restructuring mgp mgm manage rmr breit spin abp llc producer tremont northfield rmrm mandalay officer reimbursable trmt opi svc operator aic dhc empire opco portnoy sonesta adam ilpt centre jennifer person exchangeable advisor product june health device ltd pomeikang bona fruit aagh mystery vascular cardio biotechnology team mike wanshunbao ca wangshunbao wang worldwide tea cf unique mainland medical channel healthcare proton wyoming vitamin china go co california seller'

#real_estate_noise = 'million lease newmark hotel property partnership development fiscal venture tenant segment bgc september land homesale senior realogy center holding commercial group jv community sugar ffo rent reit oil joint prefer weight building dividend unit street mortgage renewal locate expand termination construction washington parking outdoor libor credit advertising repurchase held six five york fdx newly unrealized columbia wireless frp lessee mainly series stock security rental square noi foot expansion unconsolidated program industrial content nrt franchisees relocation rfg shopping cartus amendment refinance independent securitization side regional initiative affinity franchisee forma global translation cbre overview telford trg phh crisis remeasurement solution mall volume brokerage franchise agent royalty commission home driven dollar term distribution funding mffo november receivable amarra direct affiliate merger barton internalization heb kingwood drive mcfa resident developed day straight receivables acl pnc lantana santal block reinvestment entertainment member adoption asc austin master project hedge creek lot family trust multifamily phase store covid apartment asu gas water production mineral crop farm principally resort undeveloped ccsid almond pistachio trcc great ranch class produce barrel valencia approval highlight holder cfd grapevine eleven lennar rlc orchard shipyard candlestick acre entitle gateway park francisco san residential percent homesites bbxre renin bbx florida location club altman beach hospitality watersound profit pier hurricane crossing michael watercolor vacation marina origin fl sweet supplier colonial elegance northwest homesite joe southwood bvh bankruptcy pandemic county timber mae comparable fannie department gse originate mac freddie room epus competitive forbearance lodge revpar guest labor penetration adr renovation concern ff payable pursuit live compact gift transient raise disposal nasdaq beverage food cantor origination fpus unsecured caesar vici cplv iii second leverage abr propco sign execute deferral grid gla loudoun carillon extension downtown default progress la cmb quarterfield shoppes harrah east essential scope bill ceoc concession retail spread lien address redevelopment abatement multi golf shareholder exclude tax gross local margin rate exchange transaction lasalle reimburse professional asia hff msr emea banking demand consult america drove notable comparative country post pronouncement size geographically nearly etr singapore growth incremental currency pacific earn client advisory foreign billion reimbursement restructuring mgp mgm manage rmr breit spin abp llc producer tremont northfield rmrm mandalay officer reimbursable trmt opi svc operator aic dhc empire opco portnoy sonesta adam ilpt centre jennifer person exchangeable advisor product june health device ltd pomeikang bona fruit aagh mystery vascular cardio biotechnology team mike wanshunbao ca wangshunbao wang worldwide tea cf unique mainland medical channel healthcare proton wyoming vitamin china go co california seller'


In [ ]:
utilities_noise = 'idaho power pnm water gas rate million december electric pnmr asb natural plant gswc loan tnmp avista og hawaiian hei idacorp percent sjw hawaii puc nmprc group segment sjgs cpuc utc washington load montana impact ael ecova colstrip swap gaap dakota south rating committee colder ror rebate spokane graph tracker erm jpg job cut ava qf hydroelectric partly corporation corp decouple content derivative retail gross oregon nrg san jose hedge awr registrant fair groundwater land bves scvwd west impairment california asus billion military mark texas memorandum region lien drought asc unrealized asu east senior product geothermal ofc ormat complex ii jpm foreign ppas exploration iii deg opal nevada opc membership currency geo ortp orleaf ebitda tranche orpower puna recourse secure orcal electricity enable maui deposit epa oklahoma bidding banking ngls roace oahu apsc supplementary oge fhlb clause frb gathering ci dg ram ash cogeneration railcar muskogee stipulation hydropower ipuc hcc relicensing boardman opuc sensitivity sector sensitive separate settle selective severance shareholder select www scr season scrubber show saving rps roll role river shift side rise retain reversal review revise revolve repurchase statute stop stand fiscal fy roanoke save scc mvp leatherstocking pike wna midstream corn nypsc bad papuc pipeline esac mandate covid rider virginia moratorium look stockholder icc pipe pandemic warmer southgate staff pga afudc prefer mexico eta nm abandonment pvngs mod representative repeal strengthen subsidy store stadium leak steel energy westar sale thousand transmission dollar mortgage fuel creek wolf outage spp principally paper borrow aros kge network vies refuel cygne la coli bonus kansa bearing kcc eps underwriter refundings middlesex wastewater regulate tidewater jersey usa index njib njbpu srf pinelands september depsc pa main healthcare delaware lake twin amboy chod white perth marsh pwac pennsylvania sussex tesi marketable fmbs versus major consult hospital engine assurance receivables sustainable cornerstone readily preparation research name consultant advisory apparent heat engage disruption deficit receivable pwrcor city contractor actively profit york advisor subcontractor worldwide'

#utilities_noise = 'idaho power pnm water gas rate million december electric pnmr asb natural plant gswc loan tnmp avista og hawaiian hei idacorp percent sjw hawaii puc nmprc group segment sjgs cpuc utc washington load montana impact ael ecova colstrip swap gaap dakota south rating committee colder ror rebate spokane graph tracker erm jpg job cut ava qf hydroelectric partly corporation corp decouple content derivative retail gross oregon nrg san jose hedge awr registrant fair groundwater land bves scvwd west impairment california asus billion military mark texas memorandum region lien drought asc unrealized asu east senior product geothermal ofc ormat complex ii jpm foreign ppas exploration iii deg opal nevada opc membership currency geo ortp orleaf ebitda tranche orpower puna recourse secure orcal electricity enable maui deposit epa oklahoma bidding banking ngls roace oahu apsc supplementary oge fhlb clause frb gathering ci dg ram ash cogeneration railcar muskogee stipulation hydropower ipuc hcc relicensing boardman opuc sensitivity sector sensitive separate settle selective severance shareholder select www scr season scrubber show saving rps roll role river shift side rise retain reversal review revise revolve repurchase statute stop stand fiscal fy roanoke save scc mvp leatherstocking pike wna midstream corn nypsc bad papuc pipeline esac mandate covid rider virginia moratorium look stockholder icc pipe pandemic warmer southgate staff pga afudc prefer mexico eta nm abandonment pvngs mod representative repeal strengthen subsidy store stadium leak steel energy westar sale thousand transmission dollar mortgage fuel creek wolf outage spp principally paper borrow aros kge network vies refuel cygne la coli bonus kansa bearing kcc eps underwriter refundings middlesex wastewater regulate tidewater jersey usa index njib njbpu srf pinelands september depsc pa main healthcare delaware lake twin amboy chod white perth marsh pwac pennsylvania sussex tesi marketable fmbs versus major consult hospital engine assurance receivables sustainable cornerstone readily preparation research name consultant advisory apparent heat engage disruption deficit receivable pwrcor city contractor actively profit york advisor subcontractor worldwide'


In [ ]:
communication_service_list = communication_service_noise.split()
consumer_discretionary_list = consumer_discretionary_noise.split()
consumer_staples_list = consumer_staples_noise.split()
energy_list = energy_noise.split()
healthcare_list = healthcare_noise.split()
industrials_list = industrials_noise.split()
it_list = it_noise.split()
materials_list = materials_noise.split()
real_estate_list = real_estate_noise.split()
utilities_list = utilities_noise.split()

In [ ]:
all_noise = communication_service_list + consumer_discretionary_list + consumer_staples_list + energy_list + healthcare_list + industrials_list + it_list + materials_list + real_estate_list + utilities_list


In [ ]:
noise_list = pd.DataFrame(all_noise)[0].unique().tolist()

In [ ]:
len(noise_list)

2746

# preprocessing & topic modeling

In [ ]:
#create stopwords
stop_words = stopwords.words('english')
stop_words.extend(noise_list)

In [ ]:
#lemmatization
def get_pos(word):
  tag = nltk.pos_tag([word])[0][1][0].upper()
  tag_dict = {'J':wordnet.ADJ,
              'N':wordnet.NOUN,
              'V':wordnet.VERB,
              'R':wordnet.ADV}
  return tag_dict.get(tag, wordnet.NOUN)

def token_lemmatize(token):
  lemmatizer = WordNetLemmatizer()
  return lemmatizer.lemmatize(token, get_pos(token))

In [ ]:
#preprocessing
all_corpus = []
for i in range(len(df)):  
  text = df.iloc[i]['text']
  text = simple_preprocess(text)
  lemmatized = [token_lemmatize(word) for word in text if word not in stop_words]
  result = [word for word in lemmatized if word not in stop_words]
  all_corpus.append(result)

In [ ]:
#bigrams
bigram_phrases = gensim.models.Phrases(all_corpus, min_count=5, threshold=50)
bigram = gensim.models.phrases.Phraser(bigram_phrases)
bigram_corpus = [bigram[doc] for doc in all_corpus]

In [ ]:
id2word = corpora.Dictionary(bigram_corpus)
#id2word.filter_extremes(no_below=10, no_above=0.4)
train_corpus = [id2word.doc2bow(text) for text in bigram_corpus]

In [ ]:
pre_cor = []
for i in range(len(train_corpus)):
  pre_cor.append(len(train_corpus[i]))
np.mean(pre_cor), np.median(pre_cor), np.min(pre_cor), np.max(pre_cor)

(493.2086956521739, 497.0, 43, 1204)

In [ ]:
#tuning with tfidf; remove words with low tfidf(frequent appearance)
from gensim.models import TfidfModel
tfidf = TfidfModel(train_corpus, id2word=id2word)

low_value = 0.001
drop_words = [] #dropped words
for i in range(len(train_corpus)):
  bow = train_corpus[i]
  low_value_words = []
  tfidf_ids = [id for id, value in tfidf[bow]]
  bow_ids = [id for id, value in bow]
  low_value_words = [id for id, value in tfidf[bow] if value < low_value]
  words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] #words with tf-idf score 0 will be missing
  drops = low_value_words + words_missing_in_tfidf
  for item in drops:
    drop_words.append(id2word[item])
  new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
  train_corpus[i] = new_bow

In [ ]:
post_cor = []
for i in range(len(train_corpus)):
  post_cor.append(len(train_corpus[i]))
np.mean(post_cor), np.median(post_cor), np.min(post_cor), np.max(post_cor)

(473.11739130434785, 477.5, 42, 1174)

In [ ]:
#pd.DataFrame(drop_words)[0].unique().tolist()

In [ ]:
ldaModel = gensim.models.ldamodel.LdaModel(corpus=train_corpus, 
                                           id2word=id2word, 
                                           num_topics=10, 
                                           random_state=100,
                                           chunksize=100,
                                           update_every=1,
                                           passes=10)

In [ ]:
ldaModel.print_topics()

[(0,
  '0.228*"expense" + 0.034*"net" + 0.025*"increase" + 0.025*"revenue" + 0.022*"base" + 0.022*"cost" + 0.020*"include" + 0.017*"period" + 0.014*"service" + 0.014*"related"'),
 (1,
  '0.027*"cost" + 0.026*"price" + 0.017*"per" + 0.015*"include" + 0.013*"interest" + 0.012*"increase" + 0.010*"total" + 0.010*"decrease" + 0.010*"change" + 0.010*"expenditure"'),
 (2,
  '0.043*"increase" + 0.042*"revenue" + 0.029*"due" + 0.024*"cost" + 0.022*"primarily" + 0.022*"operating" + 0.021*"include" + 0.020*"change" + 0.020*"decrease" + 0.017*"business"'),
 (3,
  '0.061*"net" + 0.041*"cost" + 0.030*"operation" + 0.028*"increase" + 0.027*"decrease" + 0.022*"due" + 0.022*"compare" + 0.021*"primarily" + 0.020*"sell" + 0.016*"high"'),
 (4,
  '0.156*"year" + 0.142*"end" + 0.041*"revenue" + 0.030*"increase" + 0.026*"cost" + 0.022*"compare" + 0.021*"period" + 0.019*"decrease" + 0.017*"total" + 0.016*"loss"'),
 (5,
  '0.045*"note" + 0.040*"interest" + 0.027*"net" + 0.024*"debt" + 0.019*"amount" + 0.017*"a

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(topic_model=ldaModel, corpus=train_corpus, dictionary=id2word, mds='mmds')
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.010563  0.062321       1        1  22.224579
2      0.045276 -0.008577       2        1  15.149524
8     -0.066966 -0.022511       3        1  14.808271
4      0.042666  0.197641       4        1   9.144370
3      0.043246 -0.087973       5        1   8.494124
9     -0.118692  0.068877       6        1   7.538739
5     -0.085983 -0.181664       7        1   7.525342
1     -0.173635 -0.045919       8        1   7.083395
0      0.173566 -0.090048       9        1   5.254500
7      0.151084  0.107853      10        1   2.777155, topic_info=        Term          Freq         Total Category  logprob  loglift
450     year  30703.000000  30703.000000  Default  30.0000  30.0000
170  expense  19469.000000  19469.000000  Default  29.0000  29.0000
151      end  20915.000000  20915.000000  Default  28.0000  28.0000
70      cash  15423.000000  15423.000000  Default  27.0000  27.0000
279      net  24616.000000  24616.000000  Default  26.0000  26.0000
..       ...           ...           ...      ...      ...      ...
361  related    256.903264  17534.248977  Topic10  -5.1563  -0.6395
415    total    230.022593  13045.645143  Topic10  -5.2668  -0.4543
205     high    196.446299   7026.717401  Topic10  -5.4246   0.0067
47     asset    202.086423  21856.300504  Topic10  -5.3963  -1.0998
433      use    200.125400  15063.858390  Topic10  -5.4060  -0.7374

[818 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
5         1  0.417716  account
5         2  0.030513  account
5         3  0.168013  account
5         4  0.119734  account
5         5  0.080337  account
...     ...       ...      ...
450       4  0.747970     year
450       6  0.004658     year
450       7  0.000130     year
450       9  0.003420     year
450      10  0.243395     year

[2109 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 3, 9, 5, 4, 10, 6, 2, 1, 8])

# Logistic Regression

In [ ]:
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

In [ ]:
fin_df = pd.read_csv('fin_df.csv')
fin_df.head()

,Unnamed: 0,Company,id,market_capitalization,net_income,c&e,total_assets,total_liabilities,total_equity,current_assets,current_liabilities,total_cash_st_investment,long_term_liabilities,tobin_q,TDEBT,LTDEBT,CFIN,CNFIN,TDEBT_ast,LTDEBT_ast,CFIN_ast,CNFIN_ast,TDEBT_mcp,LTDEBT_mcp,CFIN_mcp,CNFIN_mcp,label
0,0,"Echo Global Logistics, Inc. (NasdaqGS:ECHO)",1.0,834.0,15.8,41.3,963.9,564.58,399.4,493.48,377.14,41.30,187.44,1.450960,1.413570,0.469304,0.103405,-0.840861,0.585725,0.194460,0.042847,-0.348418,0.676954,0.224748,0.049520,-0.402686,1
1,1,I-ON Digital Corp. (OTCPK:IONI),2.0,10.6,1.6,4.5,13.2,3.86,9.3,11.56,3.86,5.29,0.00,1.095455,0.415054,0.000000,0.568817,0.153763,0.292424,0.000000,0.400758,0.108333,0.364151,0.000000,0.499057,0.134906,1
2,2,"Five9, Inc. (NasdaqGM:FIVN)",3.0,11906.8,-42.1,220.4,1063.7,784.58,279.2,689.12,103.56,603.54,681.02,11.931353,2.810100,2.439183,2.161676,1.790759,0.737595,0.640237,0.567397,0.470039,0.065893,0.057196,0.050689,0.041991,1
3,3,"Medallia, Inc. (NYSE:MDLA)",4.0,5289.0,-148.7,428.3,1401.6,893.41,508.2,918.76,387.18,682.39,506.23,4.410966,1.757989,0.996124,1.342759,0.580893,0.637422,0.361180,0.486865,0.210624,0.168919,0.095714,0.129021,0.055816,1
4,4,"Cloudera, Inc.",5.0,3780.6,-162.7,298.7,2508.6,1380.67,1127.9,997.92,663.11,596.39,717.56,2.057430,1.224107,0.636191,0.528761,-0.059154,0.550375,0.286040,0.237738,-0.026597,0.365199,0.189801,0.157750,-0.017648,1


In [ ]:
fin_df['id'] = fin_df['id'].apply(int)

with topic features


In [ ]:
vecs = []
for i in range(len(df)):
  topics = ldaModel.get_document_topics(train_corpus[i], minimum_probability=0)
  topic_dist = [topics[i][1] for i in range(10)]
  vecs.append(topic_dist)

In [ ]:
vec0 = [vecs[i][0] for i in range(len(vecs))]
vec1 = [vecs[i][1] for i in range(len(vecs))]
vec2 = [vecs[i][2] for i in range(len(vecs))]
vec3 = [vecs[i][3] for i in range(len(vecs))]
vec4 = [vecs[i][4] for i in range(len(vecs))]
vec5 = [vecs[i][5] for i in range(len(vecs))]
vec6 = [vecs[i][6] for i in range(len(vecs))]
vec7 = [vecs[i][7] for i in range(len(vecs))]
vec8 = [vecs[i][8] for i in range(len(vecs))]
vec9 = [vecs[i][9] for i in range(len(vecs))]

In [ ]:
df['topic0'] = vec0
df['topic1'] = vec1
df['topic2'] = vec2
df['topic3'] = vec3
df['topic4'] = vec4
df['topic5'] = vec5
df['topic6'] = vec6
df['topic7'] = vec7
df['topic8'] = vec8
df['topic9'] = vec9

In [ ]:
def find_id(s):
  return int(''.join(x for x in s if x.isdigit()))

df['id'] = df['name'].apply(find_id)
df.head()

,name,text,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,id
0,1.txt,We are a leading provider of technology-enable...,0.163051,0.000057,0.131639,0.000057,0.000057,0.063297,0.370055,0.000880,0.270849,0.000057,1
1,10.txt,"Golden Nugget Online Gaming, Inc. (formerly kn...",0.157986,0.013913,0.005774,0.000064,0.278825,0.139234,0.268995,0.000064,0.044389,0.090755,10
2,100.txt,This annual report on Form 10-K contains forwa...,0.000043,0.000043,0.331073,0.069218,0.065904,0.000043,0.423999,0.036329,0.073308,0.000043,100
3,1001.txt,We are a global designer and manufacturer of h...,0.000040,0.008031,0.416781,0.359682,0.000040,0.008154,0.149421,0.002129,0.055681,0.000040,1001
4,1002.txt,This Management’s Discussion and Analysis of F...,0.000056,0.032080,0.176855,0.271600,0.063965,0.000056,0.013728,0.290633,0.000056,0.150972,1002


In [ ]:
topic_df = df[['id', 'topic0', 'topic1', 'topic2', 'topic3', 'topic4', 'topic5', 'topic6', 'topic7', 'topic8', 'topic9']]
topic_df.head()

,id,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9
0,1,0.163051,0.000057,0.131639,0.000057,0.000057,0.063297,0.370055,0.000880,0.270849,0.000057
1,10,0.157986,0.013913,0.005774,0.000064,0.278825,0.139234,0.268995,0.000064,0.044389,0.090755
2,100,0.000043,0.000043,0.331073,0.069218,0.065904,0.000043,0.423999,0.036329,0.073308,0.000043
3,1001,0.000040,0.008031,0.416781,0.359682,0.000040,0.008154,0.149421,0.002129,0.055681,0.000040
4,1002,0.000056,0.032080,0.176855,0.271600,0.063965,0.000056,0.013728,0.290633,0.000056,0.150972


In [ ]:
data = fin_df.merge(topic_df, on = 'id')
data

,Unnamed: 0,Company,id,market_capitalization,net_income,c&e,total_assets,total_liabilities,total_equity,current_assets,current_liabilities,total_cash_st_investment,long_term_liabilities,tobin_q,TDEBT,LTDEBT,CFIN,CNFIN,TDEBT_ast,LTDEBT_ast,CFIN_ast,CNFIN_ast,TDEBT_mcp,LTDEBT_mcp,CFIN_mcp,CNFIN_mcp,label,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9
0,0,"Echo Global Logistics, Inc. (NasdaqGS:ECHO)",1,834.0,15.8,41.30,963.9,564.58,399.4,493.48,377.14,41.30,187.44,1.450960,1.413570,0.469304,0.103405,-0.840861,0.585725,0.194460,0.042847,-0.348418,0.676954,0.224748,0.049520,-0.402686,1,0.163051,0.000057,0.131639,0.000057,0.000057,0.063297,0.370055,0.000880,0.270849,0.000057
1,1,I-ON Digital Corp. (OTCPK:IONI),2,10.6,1.6,4.50,13.2,3.86,9.3,11.56,3.86,5.29,0.00,1.095455,0.415054,0.000000,0.568817,0.153763,0.292424,0.000000,0.400758,0.108333,0.364151,0.000000,0.499057,0.134906,1,0.000224,0.000224,0.015135,0.122548,0.461988,0.000224,0.263250,0.135958,0.000224,0.000224
2,2,"Five9, Inc. (NasdaqGM:FIVN)",3,11906.8,-42.1,220.40,1063.7,784.58,279.2,689.12,103.56,603.54,681.02,11.931353,2.810100,2.439183,2.161676,1.790759,0.737595,0.640237,0.567397,0.470039,0.065893,0.057196,0.050689,0.041991,1,0.131290,0.005982,0.308105,0.000047,0.047928,0.070147,0.425506,0.010901,0.000047,0.000047
3,3,"Medallia, Inc. (NYSE:MDLA)",4,5289.0,-148.7,428.30,1401.6,893.41,508.2,918.76,387.18,682.39,506.23,4.410966,1.757989,0.996124,1.342759,0.580893,0.637422,0.361180,0.486865,0.210624,0.168919,0.095714,0.129021,0.055816,1,0.201499,0.000032,0.214969,0.000032,0.244162,0.011384,0.228055,0.000032,0.000415,0.099419
4,4,"Cloudera, Inc.",5,3780.6,-162.7,298.70,2508.6,1380.67,1127.9,997.92,663.11,596.39,717.56,2.057430,1.224107,0.636191,0.528761,-0.059154,0.550375,0.286040,0.237738,-0.026597,0.365199,0.189801,0.157750,-0.017648,1,0.105802,0.000050,0.218760,0.004836,0.142565,0.000114,0.211583,0.006270,0.068743,0.241278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,685,"RGC Resources, Inc. (NasdaqGM:RGCO)",1341,194.3,11.3,0.70,294.4,201.80,92.5,20.90,22.53,0.74,179.27,1.345448,2.181622,1.938054,0.008000,-0.235568,0.685462,0.608933,0.002514,-0.074015,1.038600,0.922645,0.003809,-0.112146,0,0.000032,0.372630,0.112990,0.000032,0.000032,0.000124,0.324326,0.120475,0.069328,0.000032
686,686,NorthWestern Corporation (NasdaqGS:NWE),1342,2949.4,155.2,5.80,6389.4,4310.35,2079.1,308.68,466.42,5.81,3843.93,1.136218,2.073181,1.848843,0.002794,-0.221543,0.674610,0.601610,0.000909,-0.072090,1.461433,1.303292,0.001970,-0.156171,0,0.000039,0.329906,0.315148,0.067570,0.000039,0.010866,0.062085,0.000678,0.015429,0.198240
687,687,"PwrCor, Inc. (OTCPK:PWCO)",1343,17.2,-0.1,0.05,0.2,0.70,-0.5,0.12,0.62,0.05,0.08,89.500000,-1.400000,-0.160000,-0.100000,1.140000,3.500000,0.400000,0.250000,-2.850000,0.040698,0.004651,0.002907,-0.033140,0,0.107992,0.045638,0.107495,0.000250,0.203611,0.000250,0.278765,0.000250,0.255500,0.000250
688,688,American States Water Company (NYSE:AWR),1344,2933.0,86.4,36.70,1791.6,1149.93,641.7,157.12,118.57,36.74,1031.36,2.278929,1.792006,1.607231,0.057254,-0.127521,0.641845,0.575664,0.020507,-0.045674,0.392066,0.351640,0.012526,-0.027900,0,0.066336,0.442608,0.117384,0.000021,0.108603,0.007150,0.215100,0.042704,0.000021,0.000073


In [ ]:
def efron_rsquared(y, y_hat):
  n = float(len(y))
  t1 = np.sum(np.power((y - y_hat), 2))
  t2 = np.sum(np.power((y - (np.sum(y) / n)), 2))
  return 1 - (t1 / t2)

In [ ]:
X = data[['topic0', 'topic1', 'topic2', 'topic3', 'topic4', 'topic5', 'topic6', 'topic7', 'topic8']].astype('float')
X = sm.add_constant(X)
y = data['label']
model = sm.Logit(y, X).fit(maxiter=10000)
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.676009
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                  label   No. Observations:                  690
Model:                          Logit   Df Residuals:                      680
Method:                           MLE   Df Model:                            9
Date:                Tue, 14 Dec 2021   Pseudo R-squ.:                 0.02473
Time:                        05:18:13   Log-Likelihood:                -466.45
converged:                       True   LL-Null:                       -478.27
Covariance Type:            nonrobust   LLR p-value:                  0.004888
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1643      0.897      1.298      0.194      -0.593       2.922
topic0        -0.2873      1.

In [ ]:
X_all = data[['tobin_q', 'market_capitalization', 'net_income', 'c&e', 'total_cash_st_investment',
              'TDEBT_mcp', 'LTDEBT_mcp', 'CFIN_mcp', 'CNFIN_mcp', 'TDEBT_ast', 'LTDEBT_ast', 'CFIN_ast', 'CNFIN_ast', 
              'topic0', 'topic1', 'topic2', 'topic3', 'topic4', 'topic5', 'topic6', 'topic7', 'topic8']].astype('float')
X_all = sm.add_constant(X_all)
y_all = data['label']
model_all = sm.Logit(y_all, X_all).fit(maxiter=100000)
print(model_all.summary())

Optimization terminated successfully.
         Current function value: 0.646951
         Iterations 2790
                           Logit Regression Results                           
Dep. Variable:                  label   No. Observations:                  690
Model:                          Logit   Df Residuals:                      669
Method:                           MLE   Df Model:                           20
Date:                Tue, 14 Dec 2021   Pseudo R-squ.:                 0.06665
Time:                        05:20:49   Log-Likelihood:                -446.40
converged:                       True   LL-Null:                       -478.27
Covariance Type:            nonrobust   LLR p-value:                 1.843e-06
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                        1.1462      0.927      1.237      0.216      -0.

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1286: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))


In [ ]:
X_all = data[['tobin_q', 'market_capitalization', 'net_income', 'c&e', 'total_cash_st_investment',
              'TDEBT_ast', 'LTDEBT_ast', 'CFIN_ast', 'CNFIN_ast', 
              'topic0', 'topic1', 'topic2', 'topic3', 'topic4', 'topic5', 'topic6', 'topic7', 'topic8']].astype('float')
X_all = sm.add_constant(X_all)
y_all = data['label']
model_all = sm.Logit(y_all, X_all).fit(maxiter=100000)
print(model_all.summary())

Optimization terminated successfully.
         Current function value: 0.657976
         Iterations 15
                           Logit Regression Results                           
Dep. Variable:                  label   No. Observations:                  690
Model:                          Logit   Df Residuals:                      672
Method:                           MLE   Df Model:                           17
Date:                Tue, 14 Dec 2021   Pseudo R-squ.:                 0.05074
Time:                        05:21:18   Log-Likelihood:                -454.00
converged:                       True   LL-Null:                       -478.27
Covariance Type:            nonrobust   LLR p-value:                 7.107e-05
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                        1.4712      0.919      1.602      0.109      -0.32

In [ ]:
X_all = data[['tobin_q', 'market_capitalization', 'net_income', 'c&e', 'total_cash_st_investment',
              'TDEBT_ast', 'LTDEBT_ast', 'CFIN_ast', 'CNFIN_ast']].astype('float')
X_all = sm.add_constant(X_all)
y_all = data['label']
model_all = sm.Logit(y_all, X_all).fit(maxiter=100000)
print(model_all.summary())

Optimization terminated successfully.
         Current function value: 0.674410
         Iterations 23
                           Logit Regression Results                           
Dep. Variable:                  label   No. Observations:                  690
Model:                          Logit   Df Residuals:                      681
Method:                           MLE   Df Model:                            8
Date:                Tue, 14 Dec 2021   Pseudo R-squ.:                 0.02703
Time:                        05:18:15   Log-Likelihood:                -465.34
converged:                       True   LL-Null:                       -478.27
Covariance Type:            nonrobust   LLR p-value:                  0.001111
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                        0.3236      0.111      2.907      0.004       0.10